In [6]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import os, glob
from utility_programs.read_routines import SAMI

In [7]:
ds_full = SAMI.auto_read('/glade/u/home/abukowski/scratch/simstorm-20110521/GITM-simstorm-run1/sami-gitm-coupled/', cols='edens', split_by_time=True, end_idx=1)

['/glade/u/home/abukowski/scratch/simstorm-20110521/GITM-simstorm-run1/sami-gitm-coupled/t110521_021500.nc']


In [11]:
coord_ds = ds_full[['glon','glat','alt']]
coord_ds = coord_ds.drop_vars(['time','mlat','mlon','malt'])
coord_ds

<xarray.Dataset>
Dimensions:  (nlt: 80, nf: 72, nz: 256)
Coordinates:
    glon     (nlt, nf, nz) float32 ...
    glat     (nlt, nf, nz) float32 ...
    alt      (nlt, nf, nz) float32 ...
Dimensions without coordinates: nlt, nf, nz
Data variables:
    *empty*

In [17]:
lat_raw = coord_ds.glat.values.copy()
lon_raw = coord_ds.glon.values.copy()
alt_raw = coord_ds.alt.values.copy()

In [18]:
lat_raw[1:3,56:58,56:58], lon_raw[1:3,56:58,56:58], alt_raw[1:3,56:58,56:58]

(array([[[-66.48, -66.39],
         [-68.57, -68.49]],
 
        [[-66.28, -66.19],
         [-68.46, -68.37]]], dtype=float32),
 array([[[291.57, 291.63],
         [290.63, 290.7 ]],
 
        [[299.61, 299.66],
         [298.92, 298.98]]], dtype=float32),
 array([[[574.17, 593.73],
         [583.06, 603.  ]],
 
        [[574.17, 593.73],
         [583.06, 603.  ]]], dtype=float32))

In [36]:
nodelon = []
nodelat = []
height = []
nodeid = np.zeros([80,72,256])

counter=0

for lt in coord_ds.nlt.values:
    for f in coord_ds.nf.values:
        for z in coord_ds.nz.values:
            nodelon.append(lon_raw[lt, f, z])
            nodelat.append(lat_raw[lt, f, z])
            height.append(alt_raw[lt, f, z])
            
            nodeid[lt, f, z] = counter
            counter += 1
        

In [82]:
vert_ids = []

didnt_work = []

known_bad = []
not_bad = []

for lt in coord_ds.nlt.values:
    for f in coord_ds.nf.values:
        for z in coord_ds.nz.values:
            
            if lt < 79:
                try:
                    vert_ids.append([nodeid[lt, f, z-1],
                                     nodeid[lt, f, z-1],
                                     nodeid[lt+1, f, z],
                                     nodeid[lt+1, f, z],
                                     nodeid[lt, f+1, z],
                                     nodeid[lt, f+1, z],
                                     nodeid[lt+1, f+1, z-1],
                                     nodeid[lt+1, f+1, z-1],])
                except IndexError:
                    didnt_work.append([lt,f,z])
                    
                except:
                    print(lt, f, z)
                    
            else:
                try:
                    vert_ids.append([nodeid[lt, f, z-1],
                                     nodeid[lt, f, z-1],
                                     nodeid[0, f, z],
                                     nodeid[0, f, z],
                                     nodeid[lt, f+1, z],
                                     nodeid[lt, f+1, z],
                                     nodeid[0, f+1, z-1],
                                     nodeid[0, f+1, z-1],])
                except IndexError:
                    didnt_work.append([lt,f,z])
                    
                except:
                    print(lt, f, z)
            
            
            if f == 71 or z == 0:
                try:
                    not_bad.append([nodeid[lt, f, z-1],
                                     nodeid[lt, f, z-1],
                                     nodeid[lt+1, f, z],
                                     nodeid[lt+1, f, z],
                                     nodeid[lt, f+1, z],
                                     nodeid[lt, f+1, z],
                                     nodeid[lt+1, f+1, z-1],
                                     nodeid[lt+1, f+1, z-1],])
                except IndexError:
                    known_bad.append([lt,f,z])
                except:
                    print(lt, f, z)

In [83]:
len(known_bad), len(not_bad)

(20551, 5609)

In [84]:
len(vert_ids), len(didnt_work)

(1454080, 20480)

In [75]:
len(known_bad), len(not_bad)

(38656, 5609)

In [76]:
len(vert_ids), len(didnt_work)

(1430295, 0)

In [77]:
not_bad[:50]

[[255.0, 255.0, 18432.0, 18432.0, 256.0, 256.0, 18943.0, 18943.0],
 [511.0, 511.0, 18688.0, 18688.0, 512.0, 512.0, 19199.0, 19199.0],
 [767.0, 767.0, 18944.0, 18944.0, 768.0, 768.0, 19455.0, 19455.0],
 [1023.0, 1023.0, 19200.0, 19200.0, 1024.0, 1024.0, 19711.0, 19711.0],
 [1279.0, 1279.0, 19456.0, 19456.0, 1280.0, 1280.0, 19967.0, 19967.0],
 [1535.0, 1535.0, 19712.0, 19712.0, 1536.0, 1536.0, 20223.0, 20223.0],
 [1791.0, 1791.0, 19968.0, 19968.0, 1792.0, 1792.0, 20479.0, 20479.0],
 [2047.0, 2047.0, 20224.0, 20224.0, 2048.0, 2048.0, 20735.0, 20735.0],
 [2303.0, 2303.0, 20480.0, 20480.0, 2304.0, 2304.0, 20991.0, 20991.0],
 [2559.0, 2559.0, 20736.0, 20736.0, 2560.0, 2560.0, 21247.0, 21247.0],
 [2815.0, 2815.0, 20992.0, 20992.0, 2816.0, 2816.0, 21503.0, 21503.0],
 [3071.0, 3071.0, 21248.0, 21248.0, 3072.0, 3072.0, 21759.0, 21759.0],
 [3327.0, 3327.0, 21504.0, 21504.0, 3328.0, 3328.0, 22015.0, 22015.0],
 [3583.0, 3583.0, 21760.0, 21760.0, 3584.0, 3584.0, 22271.0, 22271.0],
 [3839.0, 3839.0, 